In [66]:
function filter_jl_files(dir_list::Vector{String})
    jl_files = filter(x -> endswith(x, ".jl"), dir_list)
    non_jl_files = setdiff(dir_list, jl_files)
    return jl_files, non_jl_files
end

filter_jl_files (generic function with 1 method)

In [67]:
function add_filepaths!(file_dict::Dict{String, String}, dir_path::String, dir_list::Vector{String})
    for file in dir_list
        file_dict[file] = joinpath(dir_path, file)
    end
end

add_filepaths! (generic function with 1 method)

In [68]:
function find_jl_files(dir_path::String)
    file_dict = Dict{String, String}()
    jl_files, non_jl_files = filter_jl_files(readdir(dir_path))
    add_filepaths!(file_dict, dir_path, jl_files)
    new_dirs = filter(x -> isdir(joinpath(dir_path, x)), non_jl_files)
    for new_dir in new_dirs
        merge!(file_dict, find_jl_files(joinpath(dir_path, new_dir)))
    end
    return file_dict
end

find_jl_files (generic function with 1 method)

In [69]:
function filter_exp_var!(exp_and_vars::Dict{String, Dict{String, String}}, line::String)
    if contains(line, "@variable")
        var_name = get_var_name(line)
        exp_and_vars["var"][var_name] = file_name
    elseif contains(line, "@expression")
        exp_name = get_exp_name(line)
        exp_and_vars["exp"][exp_name] = file_name
    end
end

filter_exp_var! (generic function with 1 method)

In [70]:
function clean_file(dir_path::String)
    open(dir_path, "r") do file
        output_lines = ""
        for line in eachline(file)
            if !startswith(line, "#") && line != ""
                # Remove tabs
                line = replace(line, "\t" => " ")
                # Make all one line, but add space to avoid concatenating words
                output_lines = string(output_lines, " ", line)
            end
        end
        return output_lines
    end
end

clean_file (generic function with 1 method)

In [71]:
function extract_model_name(filestring::AbstractString, filter_prefix::AbstractString)
    x = split(filestring, ",")
    # Try to extract the name based on a prefix
    x2 = split(x[2], " ")
    x2 = filter(x -> startswith(x, filter_prefix), x2)
    if !isempty(x2)
        x = x2[1]
    else
        x = x[2]
    end
    # Remove any indexing
    if contains(x, "[")
        x = split(x, "[")[1]
    end
    # Remove whitespace
    return strip(x)
end

extract_model_name (generic function with 1 method)

In [72]:
function filter_filestring!(filestring::String, filter_string::String)
    cases = split(filestring, filter_string)
    outputs = Vector{String}()
    filter_prefix = string(filter_string[2])
    for i in 2:length(cases)
        push!(outputs, extract_model_name(cases[i], filter_prefix))
    end
    return outputs
end

filter_filestring! (generic function with 1 method)

In [73]:
function find_exp_var_createds(file_dict::Dict{String, String})
    # exp_and_vars = Dict{String, Dict{String, String}}("var" => Dict{String, String}(), "exp" => Dict{String, String}())
    exp_and_vars = Dict("var" => Dict{String, Any}(), "exp" => Dict{String, Any}())
    for (file_name, file_path) in file_dict
        # println(file_name)
        file_string = clean_file(file_path)
        new_var = filter_filestring!(file_string, "@variable")
        for var in new_var
            if !haskey(exp_and_vars["var"], var)
                exp_and_vars["var"][var] = Dict("created" => file_name, "accessed" => Vector{String}())
            end
            exp_and_vars["var"][var]["created"] = file_name
        end
        new_exp = filter_filestring!(file_string, "@expression")
        for exp in new_exp
            if !haskey(exp_and_vars["exp"], exp)
                exp_and_vars["exp"][exp] = Dict("created" => file_name, "accessed" => Vector{String}())
            end
            exp_and_vars["exp"][exp]["created"] = file_name
        end
    end
    return exp_and_vars
end

find_exp_var_createds (generic function with 1 method)

In [74]:
function check_access(file_dict::Dict{String, String}, target_names::Vector{String})
    file_accesses = Dict{String, Vector{String}}()
    for (file_name, file_path) in file_dict
        file_string = clean_file(file_path)
        file_accesses[file_name] = Vector{String}()
        for target_name in target_names
            if contains(file_string, target_name)
                push!(file_accesses[file_name], target_name)
            end
        end
    end
    return file_accesses
end

check_access (generic function with 1 method)

In [75]:
function get_all_accesses!(exp_and_vars::Dict{String, Dict{String, Any}}, file_dict::Dict{String, String})
    var_accessed = check_access(file_dict, collect(keys(exp_and_vars["var"])))
    for (file_name, var_names) in var_accessed
        for var_name in var_names
            push!(exp_and_vars["var"][var_name]["accessed"], file_name)
        end
    end
    exp_accessed = check_access(file_dict, collect(keys(exp_and_vars["exp"])))
    for (file_name, exp_names) in exp_accessed
        for exp_name in exp_names
            push!(exp_and_vars["exp"][exp_name]["accessed"], file_name)
        end
    end
    return exp_and_vars
end

get_all_accesses! (generic function with 1 method)

In [76]:
function make_table_header(headers::Set{String}, model_type::String)
    fun_headers = copy(headers)
    table_header = "|$(model_type) name"
    table_divider = "|:-"
    if "created" in fun_headers
        table_header = string(table_header, "|created")
        table_divider = string(table_divider, "|:-")
        delete!(fun_headers, "created")
    end    
    for header in fun_headers
        table_header = string(table_header, "|", header)
        table_divider = string(table_divider, "|", ":-")
    end
    table_header = string(table_header, "|")
    table_divider = string(table_divider, "|")
    return table_header, table_divider
end

make_table_header (generic function with 1 method)

In [77]:
function make_table_row(model_name::String, model_dict::Dict{String, Any}, headers::Set{String})
    table_row = "|$(model_name)"
    fun_headers = copy(headers)
    if "created" in fun_headers
        table_row = string(table_row, "|", model_dict["created"])
        delete!(fun_headers, "created")
    end
    for header in fun_headers
        table_row = string(table_row, "|", join(model_dict[header], ", "))
    end
    table_row = string(table_row, "|")
    return table_row
end

make_table_row (generic function with 1 method)

In [78]:
function data_2_markdown(exp_or_vars::Dict{String, Any}, model_type::String, headers::Set{String})
    table = Vector{String}()
    push!(table, "## $(model_type)s")
    table_header, table_divider = make_table_header(headers, model_type)
    push!(table, table_header)
    push!(table, table_divider)
    sorted_names = sort(collect(keys(exp_or_vars)))
    # for (name, data) in exp_or_vars
    for name in sorted_names
        push!(table, make_table_row(name, exp_or_vars[name], headers))
    end
    return table
end

data_2_markdown (generic function with 1 method)

In [79]:
# Get all Julia files in the src directory
src_path = joinpath(dirname(pwd()), "src")
file_dict = find_jl_files(src_path)

# Find where all the Variables and Expressions are defined
exp_and_vars = find_exp_var_createds(file_dict)

# Find where each Variable is accessed
exp_and_vars = get_all_accesses!(exp_and_vars, file_dict)
headers = Set{String}(["created", "accessed"])

var_table = data_2_markdown(exp_and_vars["var"], "Variable", headers)
exp_table = data_2_markdown(exp_and_vars["exp"], "Expression", headers)

open(joinpath(pwd(), "model_table.md"), "w") do io
    println(io, "# Model Variables and Expressions")
    for line in var_table
        println(io, line)
    end
    for line in exp_table
        println(io, line)
    end
end

# Find where the Objective is defined, and each of the components of the Objective